## Putting it all together

In this final web scraping lab, we'll unpickle the links we found, go to each one of them, and scrape:

- Title
- Price
- Description
- Rating
- Genre

In [1]:
#import pickle, bs4.BeautifulSoup, pandas, requests

import pickle
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
#read in the pickle file of links, from the last lab

with open('links.pkl', 'rb') as f:
    links = pickle.load(f)
    
links[:5]

['a-light-in-the-attic_1000/index.html',
 'tipping-the-velvet_999/index.html',
 'soumission_998/index.html',
 'sharp-objects_997/index.html',
 'sapiens-a-brief-history-of-humankind_996/index.html']

The links we scraped are only the last part of the page's full link. Let's reconstruct the full page link now. We'll define the base url, and then add each link from our list to the end of it.

In [3]:
#rewrite your parsing function from the last lab

def parse(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    return(soup)

In [4]:
#define the base url

base = 'http://books.toscrape.com/catalogue/'

In [5]:
#read in the first link in the list as a bs4 object, using your parse function

parse(base+links[0])


<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    A Light in the Attic | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="
    It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. Th

In [6]:
#inspect the page and pull in the title

title = parse(base + links[0]).find_all('h1')[0].text
title

'A Light in the Attic'

In [7]:
#inspect the page and pull in the price

price = parse(base + links[0]).find(class_ ='price_color').text
price

'Â£51.77'

In [8]:
#inspect the page and pull in the description

description = parse(base + links[0]).find_all('p')[3].text
description

"It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And your cradle, too?Baby, I think someone down here'sGot it in for you. Shel, you never sounde

In [9]:
#inspect the page and pull in the rating

rating = parse(base + links[0]).find_all(class_ = 'star-rating')[0]['class'][1]
rating

'Three'

In [10]:
#inspect the page and pull in the genre

genre = parse(base + links[0]).find_all(class_ ='breadcrumb')[0].find_all('a')[2].text
genre

'Poetry'

In [11]:
#loop through every link and save each feature into a dataframe, along with the link for the respective page

scraped_data = []

for i in range(0, len(links)):
    soup = parse(base + links[i])
    title = soup.find_all('h1')[0].text
    price = soup.find(class_ ='price_color').text
    description = soup.find_all('p')[3].text
    rating = soup.find_all(class_ = 'star-rating')[0]['class'][1]
    genre = soup.find_all(class_ ='breadcrumb')[0].find_all('a')[2].text
    link = links[i]

    row = {
        'Title' : title,
        'Price' : price,
        'Description' : description,
        'Rating' : rating,
        'Link' : link,
        'Genre' : genre,
    }
    scraped_data.append(row)

my_df = pd.DataFrame(scraped_data)
print('Scraped {} pages.'.format(len(my_df)))

Scraped 1000 pages.


In [12]:
#show the top of your dataframe to confirm everything came through correctly

my_df.head()

,Title,Price,Description,Rating,Link,Genre
0,A Light in the Attic,Â£51.77,It's hard to imagine a world without A Light i...,Three,a-light-in-the-attic_1000/index.html,Poetry
1,Tipping the Velvet,Â£53.74,"""Erotic and absorbing...Written with starling ...",One,tipping-the-velvet_999/index.html,Historical Fiction
2,Soumission,Â£50.10,"Dans une France assez proche de la nÃ´tre, un ...",One,soumission_998/index.html,Fiction
3,Sharp Objects,Â£47.82,"WICKED above her hipbone, GIRL across her hear...",Four,sharp-objects_997/index.html,Mystery
4,Sapiens: A Brief History of Humankind,Â£54.23,From a renowned historian comes a groundbreaki...,Five,sapiens-a-brief-history-of-humankind_996/index...,History


In [13]:
#save your dataframe as a .csv

my_df.to_csv('scraped_books.csv')